# In this notebook, we will train a CNN model to classify between the car images we gathered after validating the labels our algorithm generated 

In [ ]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import os

# Loading the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
original_car_data = np.load('/content/drive/MyDrive/Colab Notebooks/full_image_data.npy')
original_car_labels = np.load('/content/drive/MyDrive/Colab Notebooks/full_label_data.npy')
stanford_car_data = np.load('/content/drive/MyDrive/full_image_data_stanford.npy')
stanford_car_labels = np.load('/content/drive/MyDrive/full_label_data_stanford.npy')
berkley_car_data = np.load('/content/drive/MyDrive/Copy of full_image_data_berkley.npy')
berkley_car_labels = np.load('/content/drive/MyDrive/Copy of full_label_data_berkley.npy')

In [ ]:
original_car_data = list(original_car_data)
stanford_car_data = list(stanford_car_data)
berkley_car_data = list(berkley_car_data)

In [ ]:
for i in range(len(original_car_data)):
    original_car_data[i] = tf.image.resize(original_car_data[i], (128, 128))
     
for i in range(len(stanford_car_data)):
    stanford_car_data[i] = tf.image.resize(stanford_car_data[i], (128, 128))

for i in range(len(berkley_car_data)):
    berkley_car_data[i] = tf.image.resize(berkley_car_data[i], (128, 128))
    berkley_car_data[i] = berkley_car_data[i] / 255.0

In [ ]:
original_car_data = np.array(original_car_data)
stanford_car_data = np.array(stanford_car_data)
berkley_car_data = np.array(berkley_car_data)

## Since we have 8 classes to classify, we'll use stratified split to make sure a roughly equal amount of each class is in each set (training, validation and testing).

## We're performing the split on each dataset sperately (original segmentation images, stanford cars dataset and pictures of cars from berkley deep drive) so that we can check performance across datasets, however training was performed on all datasets combined.

In [ ]:
# separate each of them train-test-validation

# original_car_data:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)
for train_index, test_index in sss.split(original_car_data, original_car_labels):  
    X_train_original, X_test_original = original_car_data[train_index], original_car_data[test_index]
    y_train_original, y_test_original = original_car_labels[train_index], original_car_labels[test_index]

sss = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)
for train_index, test_index in sss.split(X_train_original, y_train_original):  
    X_train_original, X_val_original = X_train_original[train_index], X_train_original[test_index]
    y_train_original, y_val_original = y_train_original[train_index], y_train_original[test_index]

print('Shape of original X_train is:', X_train_original.shape)
print('Shape of original y_train is:', y_train_original.shape)
print('Shape of original X_test is:', X_test_original.shape)
print('Shape of original y_test is:', y_test_original.shape)
print('Shape of original X_val is:', X_val_original.shape)
print('Shape of original y_val is:', y_val_original.shape)
print('\n')

# stanford car data:
sss = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)
for train_index, test_index in sss.split(stanford_car_data, stanford_car_labels):  
    X_train_stanford, X_test_stanford = stanford_car_data[train_index], stanford_car_data[test_index]
    y_train_stanford, y_test_stanford = stanford_car_labels[train_index], stanford_car_labels[test_index]

sss = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)
for train_index, test_index in sss.split(X_train_stanford, y_train_stanford):  
    X_train_stanford, X_val_stanford = X_train_stanford[train_index], X_train_stanford[test_index]
    y_train_stanford, y_val_stanford = y_train_stanford[train_index], y_train_stanford[test_index]

print('Shape of stanford X_train is:', X_train_stanford.shape)
print('Shape of stanford y_train is:', y_train_stanford.shape)
print('Shape of stanford X_test is:', X_test_stanford.shape)
print('Shape of stanford y_test is:', y_test_stanford.shape)
print('Shape of stanford X_val is:', X_val_stanford.shape)
print('Shape of stanford y_val is:', y_val_stanford.shape)
print('\n')

# berkley car data:
sss = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)
for train_index, test_index in sss.split(berkley_car_data, berkley_car_labels):  
    X_train_berkley, X_test_berkley = berkley_car_data[train_index], berkley_car_data[test_index]
    y_train_berkley, y_test_berkley = berkley_car_labels[train_index], berkley_car_labels[test_index]

sss = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)
for train_index, test_index in sss.split(X_train_berkley, y_train_berkley):  
    X_train_berkley, X_val_berkley = X_train_berkley[train_index], X_train_berkley[test_index]
    y_train_berkley, y_val_berkley = y_train_berkley[train_index], y_train_berkley[test_index]

print('Shape of berkley X_train is:', X_train_berkley.shape)
print('Shape of berkley y_train is:', y_train_berkley.shape)
print('Shape of berkley X_test is:', X_test_berkley.shape)
print('Shape of berkley y_test is:', y_test_berkley.shape)
print('Shape of berkley X_val is:', X_val_berkley.shape)
print('Shape of berkley y_val is:', y_val_berkley.shape)

Shape of original X_train is: (318, 128, 128, 3)
Shape of original y_train is: (318,)
Shape of original X_test is: (100, 128, 128, 3)
Shape of original y_test is: (100,)
Shape of original X_val is: (80, 128, 128, 3)
Shape of original y_val is: (80,)


Shape of stanford X_train is: (260, 128, 128, 3)
Shape of stanford y_train is: (260,)
Shape of stanford X_test is: (82, 128, 128, 3)
Shape of stanford y_test is: (82,)
Shape of stanford X_val is: (65, 128, 128, 3)
Shape of stanford y_val is: (65,)


Shape of berkley X_train is: (329, 128, 128, 3)
Shape of berkley y_train is: (329,)
Shape of berkley X_test is: (103, 128, 128, 3)
Shape of berkley y_test is: (103,)
Shape of berkley X_val is: (83, 128, 128, 3)
Shape of berkley y_val is: (83,)


In [ ]:

# concatenate all x_trains and all x_tests
X_train = np.concatenate([X_train_original, X_train_stanford, X_train_berkley])
X_test = np.concatenate([X_test_original, X_test_stanford, X_test_berkley])
y_train = np.concatenate([y_train_original, y_train_stanford, y_train_berkley])
y_test = np.concatenate([y_test_original, y_test_stanford, y_test_berkley])
X_val = np.concatenate([X_val_original, X_val_stanford, X_val_berkley])
y_val = np.concatenate([y_val_original, y_val_stanford, y_val_berkley])
print('Shape of X_train is:', X_train.shape)
print('Shape of X_test is:', X_test.shape)
print('Shape of Y_train is:', y_train.shape)
print('Shape of Y_test is:', y_test.shape)
print('Shape of X_val is:', X_val.shape)
print('Shape of Y_val is:', y_val.shape)


Shape of X_train is: (907, 128, 128, 3)
Shape of X_test is: (285, 128, 128, 3)
Shape of Y_train is: (907,)
Shape of Y_test is: (285,)
Shape of X_val is: (228, 128, 128, 3)
Shape of Y_val is: (228,)


In [ ]:
classification_dict = {
     0 : 'front',
     1 : 'front-right',
     2 : 'right',
     3 : 'back-right',
     4 : 'back' ,
     5 : 'back-left',
     6 : 'left' ,
     7 : 'front-left'
}
rev_classification_dict = {v : k for k, v in classification_dict.items()}
rev_classification_dict

{'front': 0,
 'front-right': 1,
 'right': 2,
 'back-right': 3,
 'back': 4,
 'back-left': 5,
 'left': 6,
 'front-left': 7}

## After alot of experimentation, we found that pretty heavy regularization improved validation set performance significantly. On most layers, BatchNorm performed better than Dropout, and around 25-35 epochs we're needed before overfitting. The best accuracy we managed to reach is around 78.95%.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Input, Conv2D, MaxPooling2D, AveragePooling2D, Dropout, BatchNormalization

conv_model = Sequential()
conv_model.add(Input(shape=(128, 128, 3)))

conv_model.add(Conv2D(32, kernel_size=(3, 3), activation="relu"))
conv_model.add(MaxPooling2D(pool_size=(2, 2)))
conv_model.add(BatchNormalization())

conv_model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
conv_model.add(MaxPooling2D(pool_size=(2, 2)))
conv_model.add(BatchNormalization())

conv_model.add(Conv2D(128, kernel_size=(3, 3), activation="relu")) 
conv_model.add(MaxPooling2D(pool_size=(2, 2))) 
conv_model.add(BatchNormalization())

conv_model.add(Conv2D(256, kernel_size=(3, 3), activation="relu")) 
conv_model.add(MaxPooling2D(pool_size=(2, 2))) 
conv_model.add(BatchNormalization())

conv_model.add(Conv2D(512, kernel_size=(3, 3), activation="relu")) 
conv_model.add(MaxPooling2D(pool_size=(2, 2)))  

conv_model.add(Flatten())
conv_model.add(Dropout(0.5))
conv_model.add(Dense(8, activation="softmax"))

conv_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
conv_model.fit(X_train, y_train, validation_data = (X_val, y_val), batch_size = 10, epochs = 25)

Epoch 1/25
91/91 [==============================] - 2s 15ms/step - loss: 5.3269 - accuracy: 0.2536 - val_loss: 2.6417 - val_accuracy: 0.1228
Epoch 2/25
91/91 [==============================] - 1s 12ms/step - loss: 4.7448 - accuracy: 0.3363 - val_loss: 4.7744 - val_accuracy: 0.0658
Epoch 3/25
91/91 [==============================] - 1s 12ms/step - loss: 4.3075 - accuracy: 0.3804 - val_loss: 3.8436 - val_accuracy: 0.1579
Epoch 4/25
91/91 [==============================] - 1s 11ms/step - loss: 3.5436 - accuracy: 0.4598 - val_loss: 1.7974 - val_accuracy: 0.4868
Epoch 5/25
91/91 [==============================] - 1s 11ms/step - loss: 2.7016 - accuracy: 0.5160 - val_loss: 2.2333 - val_accuracy: 0.4342
Epoch 6/25
91/91 [==============================] - 1s 12ms/step - loss: 1.8607 - accuracy: 0.5843 - val_loss: 1.4045 - val_accuracy: 0.6096
Epoch 7/25
91/91 [==============================] - 1s 11ms/step - loss: 1.3479 - accuracy: 0.6560 - val_loss: 1.0947 - val_accuracy: 0.6140
Epoch 8/25
91

# We'll save the final model and check it's overall results in the DL Project model results notebook.